In [2]:
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=True)

[WDM] - Downloading: 100%|██████████| 6.79M/6.79M [00:00<00:00, 13.8MB/s]


In [2]:
url1 = "C:/Users/freew/Downloads/Waffle House Stores In "
url2 = ", United States.html"

states = ["Alabama","Arizona","Arkansas","Colorado","Delaware","Florida","Georgia","Illinois","Indiana","Kansas","Kentucky","Louisiana",
          "Maryland","Mississippi","Missouri","New Mexico","North Carolina","Ohio","Oklahoma","Pennsylvania","South Carolina","Tennessee",
          "Texas","Virginia","West Virginia"]

waffle_house = []

for state in states:
    browser.visit(url1 + state + url2)
    soup = BeautifulSoup(browser.html, 'lxml')

    results = soup.find_all('div', class_="tiles wider")

    for result in results:
        # Extract only text from Waffle House title
        titles = result.find_all('span', class_='name')
        title = [item.getText(strip=True) for item in titles]
        
        # Extract only text from Waffle House address 
        addresses = result.find_all('span', class_="address")
        address = [item.getText(strip=False) for item in addresses]

        char_remove = ["\n", "  "]
        for char in char_remove:
            address = [a.replace(char,'%') for a in address]

        address = [a.replace('%','') for a in address]


    for i in range(len(title)):
        temp_dict = {
            "title": title[i],
            "address": address[i]
        }
        waffle_house.append(temp_dict)
    
waffle_house_df1 = pd.DataFrame.from_dict(waffle_house)

waffle_house_df1.to_csv("waffle_house_addresses.csv")

In [24]:
import requests
from api_keys import gkey

waffle_house_df1 = pd.read_csv("waffle_house_addresses.csv")

waffle_house = []

g_url = "https://maps.googleapis.com/maps/api/geocode/json?"
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index, row in waffle_house_df1.iterrows():
    print(f"Searching: {index} of {len(waffle_house_df1)} for {row['title']} {row['address']}")
    params = {
        "key": gkey,
        "address": row['address']
    }

    coordinates = requests.get(g_url, params).json()

    latlng = f"{coordinates['results'][0]['geometry']['location']['lat']}, {coordinates['results'][0]['geometry']['location']['lng']}"

    params = {
        "radius": 10000,
        "keyword": "Waffle House",
        "key": gkey,
        "location": latlng
    }

    # Make and API request using the params dictionary
    waffle_house_r = requests.get(base_url, params).json()

    for i in range(len(waffle_house_r['results'])):
        try:
            temp_dict = {
                "lat": waffle_house_r['results'][i]['geometry']['location']['lat'],
                "lng": waffle_house_r['results'][i]['geometry']['location']['lng'],
                "rating": waffle_house_r['results'][i]['rating'],
                "place_id": waffle_house_r['results'][i]['place_id'],
                "total_ratings": waffle_house_r['results'][i]['user_ratings_total'],
                "plus_code": waffle_house_r['results'][i]['plus_code']['compound_code'],
                "vicinity": waffle_house_r['results'][i]['vicinity'],
                "api_name": waffle_house_r['results'][i]['name'],
                "index": index,
                "address": row['address'],
                "name": row['title']
            }
        except:
            print("Invalid results returned!")
    
        waffle_house.append(temp_dict)

Searching: 0 of 1972 for Waffle House #1004 1900 COMMERCE ST CULLMAN, AL 35055 US
Searching: 1 of 1972 for Waffle House #1013 183 HIGHWAY 43 SOUTH TUSCUMBIA, AL 35674 US
Searching: 2 of 1972 for Waffle House #1033 1440 GADSDEN HWY BIRMINGHAM, AL 35235 US
Searching: 3 of 1972 for Waffle House #1035 2167 S. COLLEGE ST. AUBURN, AL 36830 US
Searching: 4 of 1972 for Waffle House #1043 2442 SOUTH OATES ST DOTHAN, AL 36301 US
Searching: 5 of 1972 for Waffle House #1093 1850 ASHVILLE RD LEEDS, AL 35094 US
Searching: 6 of 1972 for Waffle House #1113 7031 THEODORE DAWES THEODORE, AL 36582 US
Searching: 7 of 1972 for Waffle House #1151 5419 HWY 280 SOUTH BIRMINGHAM, AL 35244 US
Searching: 8 of 1972 for Waffle House #1165 5448 US HWY 90 W MOBILE, AL 36619 US
Searching: 9 of 1972 for Waffle House #1166 1738 OPELIKA RD AUBURN, AL 36830 US
Searching: 10 of 1972 for Waffle House #1179 80 ALABAMA HWY 75 N ALBERTVILLE, AL 35951 US
Searching: 11 of 1972 for Waffle House #1184 5037 OSCAR BAXTER DR TUSCALO

In [30]:
# Temporary Code to search one address
# g_url = "https://maps.googleapis.com/maps/api/geocode/json?"
# base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# params = {
#     "key": gkey,
#     "address": "1900 COMMERCE ST CULLMAN, AL 35055 US"
# }

# coordinates = requests.get(g_url, params).json()

# latlng = f"{coordinates['results'][0]['geometry']['location']['lat']}, {coordinates['results'][0]['geometry']['location']['lng']}"

# params = {
#     "radius": 5000,
#     "keyword": "Waffle House",
#     "key": gkey,
#     "location": latlng
# }

#     # Make and API request using the params dictionary
# waffle_house_r = requests.get(base_url, params).json()


In [123]:
import numpy as np
import json

waffle_house_df2 = pd.DataFrame(waffle_house)

waffle_house_df2['temp1'] = waffle_house_df2['address'].str.split(pat = ' ',n=1) 
waffle_house_df2['temp2'] = waffle_house_df2['vicinity'].str.split(pat = ' ',n=1) 
waffle_house_df2['temp3'] = waffle_house_df2['index']
waffle_house_df2['keep1'] = waffle_house_df2.agg('{0[temp1][0]} {0[temp3]}'.format, axis=1)
waffle_house_df2['keep2'] = waffle_house_df2.agg('{0[temp2][0]} {0[temp3]}'.format, axis=1)
waffle_house_df2['keep3'] = np.where(waffle_house_df2['keep1'] ==waffle_house_df2['keep2'], True, False)

waffle_house_df2.drop(['temp1','temp2','temp3','keep1','keep2'], axis=1, inplace=True)

waffle_house_final = waffle_house_df2[waffle_house_df2['keep3']==True].reset_index()
waffle_house_final.drop(['keep3','level_0','index'], axis=1, inplace=True)

# Create CSV file
waffle_house_final.to_csv("waffle_house.csv")

# Create JSON file 
result = waffle_house_final.to_json(orient="index")
parsed = json.loads(result)

with open('waffle_house.json', 'w' ) as f:
    json.dump(parsed, f, ensure_ascii=False, indent=4)

